# Laravel 4 : Middleware

## Introduction

La notion de « _middleware_ » (ou intergiciel en français) est souvent définie de manière confuse. Si l'on en croit [Wikipedia](https://www.wikiwand.com/fr/Intergiciel) :

> En architecture informatique, un middleware (anglicisme) ou intergiciel est un logiciel tiers qui crée un réseau d'échange d'informations entre différentes applications informatiques. Le réseau est mis en œuvre par l'utilisation d'une même technique d'échange d'informations dans toutes les applications impliquées à l'aide de composants logiciels.
>
>Les composants logiciels du middleware assurent la communication entre les applications quels que soient les ordinateurs impliqués et quelles que soient les caractéristiques matérielles et logicielles des réseaux informatiques, des protocoles réseau, des systèmes d'exploitation impliqués.
>
>Les techniques les plus courantes d'échange d'informations sont l'échange de messages, l'appel de procédures à distance et la manipulation d'objets à distance.
>
>Les middlewares sont typiquement utilisés comme ciment pour relier des applications informatiques disparates des systèmes d'information des entreprises et des institutions.

C'est donc une brique logicielle qui n'a pas de caractère _fonctionnel_, mais qui permet, en particulier dans les applications distribuées, d'établir des communications entre différents élements, éventuellement hétérogènes, d'un _cluster_.

Cette technique a aussi été utlisée comme adaptateur, pour permettre à des applications « modernes » d'interagir avec une base de code ancienne ( « _legacy_ » comme disent les anglophones.

Ce qui va être décrit ci-après ne correspond pas tout à fait à la définition, mais s'en rapproche dans le sans où les classes intergicielles de Laravel ne s'occupent pas du traitement des données lui-même (dévolu au **contrôleur**), mais vont s'occuper de tâches plus « administratives », comme les droits d'accès aux ressources, par exemple. On peut voir cela comme une couche transparente entre le routeur et le contrôleur, qu ne modifie pas les données mais établit la validité de la requête.

Le _middelware_ fournit un mécanisme pratique pour inspecter et filtrer les requêtes HTTP qui entrent dans votre application. Par exemple, Laravel inclut un intergiciel qui vérifie que l'utilisateur de votre application est authentifié. Si l'utilisateur n'est pas authentifié, l'intergiciel le redirigera vers l'écran de connexion de votre application. Cependant, si l'utilisateur est authentifié, l'intergiciel permettra à la requête de se poursuivre dans l'application.

Des intergiciels supplémentaires peuvent être écrits pour effectuer une variété de tâches autres que l'authentification. Par exemple, un intergiciel de journalisation peut enregistrer toutes les demandes entrantes dans votre application. Plusieurs intergiciels sont inclus dans le framework Laravel, notamment des intergiciels pour l'authentification et la protection CSRF. Tous ces intergiciels sont situés dans le répertoire `app/Http/Middleware`.

## Structure d'une classe intergicielle

Toutes les classes intergicielles héritent normalement de la classe `Illuminate\Auth\Middleware\Authenticate`, mais Laravel cherche des classes qui sont dans le « bon » dossier (`app/Http/Middleware`).

Une telle classe repose essentiellement sur une méthode nommée `handle`. Si nous regardons la classe de Laravel, nous voyons que sa signature estla suivante :

In [ ]:
%%php

public function handle($request, Closure $next, ...$guards);

Pour expliquer très succinctement la signature de la fonction :

1. Vous voyons que le premier paramètre de la méthode (`request`) attend la requête reçue par le serveur (on s'attendrait d'ailleurs à ce que ce paramètre soit typé). L'objet de la classe `Request` est construit pas le routeur et « intercepté » par la couche intergicielle avant d'être, éventuellement, transmis au contrôleur.
2. Un deuxième paramètre (`next`) attend une fonction. Celle-ci représente un traitement _suivant_. En effet, il faut comprendre que la couche intergicielle est formée de plusieurs sous-couches, qui sont appelées successivement pour effectuer toutes les tâches préalables au traitement de la requête proprement dit. La couche et l'enchaînement des différentes tâches et gérée par Laravel.
3. Le troisième paramètre, (`guards`), sert à définir des modes d'authentification pour les utilisateurs. Cette notion de « garde » est liéeà celles de « _driver_ » (pilote) et « _provider_ » (fournisseur, généralement une classe d'utilisateurs).

A titre d'exemple, voici la méthode `handle` de la classe intergiceille `RedirectIfAuthenticated` :

In [ ]:
%%php

public function handle(Request $request, Closure $next, ...$guards)
{
        $guards = empty($guards) ? [null] : $guards;

        /*
         * On fait appel à tous les gardes pour voir si une méthode d’authentification réussit
         * Dès qu’un succès est constaté, l’utilisateur est renvoyé vers la page d’accueil
         */
        foreach ($guards as $guard) {
            if (Auth::guard($guard)->check()) {
                return redirect(RouteServiceProvider::HOME);
            }
        }

        /*
         * Sinon, en cas d’échec, on passe à l’élément suivant de la chaîne
         */
        return $next($request);
    }

> <div class="alert alert-block alert-info">
> <b>Note :</b> La fonction `redirect` employée ici est un utilitaire dont on peut trouver l'implémentation dans la bibliothèque `/laravel/framework/src/Illuminate/foundation/helpers.php`
> </div>

## Utiliser les classes intergicielles

### Classes globales

Si vous voulez qu'une tâche soit exécutée pour toute requête HTTP arrivantsur le serveur, le moyen le plus simple est de l'enregistrer dans le fichier `app/Http/Kernel.php`.

Cette classe définit plusieurs tableaux dont un appelé `middleware`, qui liste les tâches globales.

In [ ]:
%%php

protected $middleware = [
    // \App\Http\Middleware\TrustHosts::class,
    \App\Http\Middleware\TrustProxies::class,
    \Fruitcake\Cors\HandleCors::class,
    \App\Http\Middleware\PreventRequestsDuringMaintenance::class,
    \Illuminate\Foundation\Http\Middleware\ValidatePostSize::class,
    \App\Http\Middleware\TrimStrings::class,
    \Illuminate\Foundation\Http\Middleware\ConvertEmptyStringsToNull::class,
];

### Assigner une classe à une route

Si une route spécifique doit passer par certains pré-traitements, il existe alors deux syntaxes pour cela.

La plus simple est d'associer la classe à la route avec la méthode `middleware`.

In [ ]:
use App\Http\Middleware\EnsureTokenIsValid;
 
Route::get('/profile', function () { /* ... */ })->middleware(EnsureTokenIsValid::class);

Il est aussi possible de nommer la classe en l'enregistrant dans le tableau `routeMiddleware` du fichier `app/Http/Kernel.php`. Dans ce cas, si notre classe correpond àla clef `validToken`, la syntaxe devient plus simple :

In [ ]:
Route::get('/profile', function () { /* ... */ })->middleware('validToken');

Si une classe doit être utilisée pour plusieurs routes, cette syntaxe facilite la réutilisation.

### Assigner une classe à un groupe de routes

Comme les routes peuvent être groupées (cf. [Routage]()), la méthode middleware peut aussi être utilisée une seule fois, comme par exemple :

In [ ]:
%%php

public function boot()
{
    $this->configureRateLimiting();

    $this->routes(function () {
        Route::prefix('api')
            /*
             * Pour toutes les routes du groupe 'api'
             * appliquer la tâche définie sous le nom 'api' dans le tableau `routeMiddleware'
             */
            ->middleware('api')
            ->namespace($this->namespace)
            ->group(base_path('routes/api.php'));

        /*
         * Comme les routes 'web' ne sont pas préfixées,
         * on peut utiliser directement la méthode 'Route::middleware' sur ce groupe
         */
        Route::middleware('web')
            ->namespace($this->namespace)
            ->group(base_path('routes/web.php'));
    });
}

### Exclure une tâche

Si une tâche est assignée à ungroupe de routes, il arrive que l'on souhaite la désactiver dans certains cas. Cela est possible avec la méthode `withoutMiddleware`: 

In [ ]:
use App\Http\Middleware\EnsureTokenIsValid;
 
Route::get('/profile', function () { /* ... */ })->withoutMiddleware('validToken');

On ne peut exclure des tâches enregistrées comme globales.

### Groupes de classes intergicielles

Dans de nombreux cas, une série de tâches doivent être effectuées, selon le contexte de l'application. Il est alors possible de grouper ces tâches au sein du tableau `middlewareGroups` du fichier `Kernel.php`. Comme nous pouvons le voir, des groupes par défuat sont déjà définis, `web`et `api`, qui correspondant aux contextes des fichiers `web.php`et `api.php` du dossier `/routes`. La concordance est configurée dans la classe `RouteServiceProvider`.

Appliquer un groupe d'intergiciels à une route ou à groupe de routes sefait suivant la même syntaxe que celle vue précédemment :

In [ ]:
%%php

// Pour une route
Route::get('/profile', function () { /* ... */ })->middleware('web');

// Pour un groupe de route
Route::middleware('web')->group(base_path('routes/web.php'));


### Paramètres pour les classes intergicielles

Il est possible qu'une tâche nécessite un paramètre.

Dans ce cas, la valeur de l'argument est accolé au nom de l'intergiciel avec le caractères `: `.

In [ ]:
%%php

Route::put('/post/{id}', function ($id) { /* .. */ })->middleware('role:editor');

Dans cet exemple, la création d'articles n'est autorisée quepour des personnes ayant le rôle « éditeur** ».

Pour que cette valeur soit prise en compte par la classe, il est nécessaire d'introdure un nouveau paramètre dans la méthode `handle`de la classe correspondants :

In [ ]:
%%php

public function handle($request, Closure $next, $role)
{
    if (! $request->user()->hasRole($role)) {
        // Redirect...
    }

    return $next($request);
}

## Middleware, gardes et authentification

